In [1]:
#!pip install numpy

In [2]:
#!pip install matplotlib

In [3]:
#!pip install Cython

In [4]:
#!pip install pandas

In [5]:
#conda install -c conda-forge cartopy

In [6]:
#!pip install netcdf4

In [7]:
#!pip install tensorflow

In [8]:
#!pip install -U scikit-learn

In [9]:
#Manejo de Datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Imagenes satelitales
import cartopy.crs as ccrs
import cartopy.feature as cf
from netCDF4 import Dataset, num2date

#Machine learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

#Librerias estandar (Extras)
import re
import os
import time
import random

In [10]:
#conda uninstall cudnn

In [11]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [12]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17731649908387155228
xla_global_id: -1
]


In [37]:
#El modelo solo considera en input_shape(x,x,1), el 1 se puede cambiar para abarcar mas canales de imagenes satelitales
def crearModelo(W,H,dim, output):
    print(f"Se creo un modelo con input ({W,H,dim}) y output({output})")
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(W, H, dim)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(output))
    
    print(model.summary())
    return model


In [53]:
#Se le da un tensor de 4 dimensiones
#[0] =  dato de precipitacion
#[1] = Punto de la estacion (Longitud)
#[2] = Punto de la estacion (Latitud)
#[3] = Fecha (año-mes-dia-hora)

#Devuelve x,y
#X = Dato de precipitacion
#Y = np.Array de las matrices de colores de cada producto en products (C08,C07 o C13)
def leerImagenArea(tensor, umbral, path_base,margen,products): 
    
    """
    -----------------------------------------------------------------------------------    
    !!!!!VERIFICAR QUE LA HORA DE LA IMAGEN SATELITAL SEA IGUAL A LA HORA PERU!!!!!!!!
    -----------------------------------------------------------------------------------
    
    Los archivos se deben encontrar en carpetas ordenadas : ../GOES/{producto}/{año}/{mes}/{ARCHIVO}.nc
    ARCHIVO = G16_{producto}_Cyl_{año}{mes}{dia}-{hora}00.nc'
    
    EJEMPLO : path_base + GOES/C8/2019/02/G16_C08_Cyl_20190210-1600.nc
    """
    
    #Se define por defecto el path base - (Temporal)
    #path_base  =  'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/'
    #path_base  =  '../GOES'
    
    try:
        #Fecha = 2019 01 05 22
        fecha = str(tensor.numpy()[3].decode('UTF-8'))
        year,month,day,hour = fecha.split('-')
        
   
    except:
        print("No se pudo leer la fecha")
        print(tensor.numpy()[3].decode('UTF-8'))
        return -1
    
    
    #El ancho y alto sera el margen que se dara desde el punto de origen (estacion)
    #Esta en decimales (1 decimal == 100Km) - (Temporal)
    alto= margen[0]
    ancho= margen[1]
    
    
    #Se define el producto 
    mapaArrays = []
    for product in products:    
        origen = [float(tensor.numpy()[1].decode('UTF-8')),float(tensor.numpy()[2].decode('UTF-8'))]    
        filename = f'{path_base}/{product}/{year}/{month}/G16_{product}_Cyl_{year}{month}{day}-{hour}00.nc'    
        try:
            ds = Dataset(filename)
        except:
            print("No se pudo leer el archivo")
            print(filename)
            return -1

        # convierte el tiempo de formato numerico a formato fecha y hora de python
        #date = num2date(ds.variables['time'][:], ds.variables['time'].units, only_use_cftime_datetimes=False, only_use_python_datetimes=True)

        # convierte el formato de la variable de Int16 a Float32 y guarda el resultado
        field = ds.variables['CMI'][:].data.astype(np.float32)/100.0

        # obtiene las coordenadas de los pixeles
        lon = ds.variables['longitude'][:].data
        lat = ds.variables['latitude'][:].data    

        #Se define el margen para recortar la imagen satelital
        maxLon=origen[0]+ancho
        minLon=origen[0]-ancho
        maxLat=origen[1]+alto
        minLat=origen[1]-alto

        #Booleanos que ayudarán a buscar el margen
        altoMin = False
        altoMax = False


        #Inicializamos los "indices"
        lom = 0
        loM = 0
        lam = 0
        laM = 0

        """
        Tener en cuenta que el arreglo de longitudes (lon) esta ordenado de manera creciente,
        mientras que el de latitudes (lat) esta de manera decreciente
        """    
        for i in range(0,len(lon)):
            if lon[i]>=minLon and not altoMin:
                altoMin = True
                lom = i
            if lon[i]<=maxLon:
                loM = i

        for j in range(0,len(lat)):
            if lat[j]>=minLat:    
                laM = j
            if lat[j]<=maxLat and not altoMax:
                altoMax = True
                lam = j   
                
        mapaArrays.append(field[lam:laM,lom:loM])
    
    dato = float(tensor.numpy()[0].decode('UTF-8'))
    if dato > umbral:
        dato  = 1
    else: 
        dato = 0   
    
    if len(products) == 1:
        return mapaArrays[0], dato
    
    return np.dstack(mapaArrays), dato
       

In [15]:
#Devuelve una lista con lo indices que no se encontraron lso archivos y el producto
#Servira para ver si se teinen todas las imagenes necesarias para el entrenamiento
def comprobarFile(df,products,path_base):        
    no_index = []
    no_product = []
    for i in df.index:       
        year,month,day,hour = df['fecha'][i].split('-')
        tmpProduct = []        
        for p in products:
            filename = f'{path_base}/{p}/{year}/{month}/G16_{p}_Cyl_{year}{month}{day}-{hour}00.nc'       
            existe = os.path.exists(filename)
            if not existe:
                tmpProduct.append(p)
        if len(tmpProduct)>0:
            no_index.append(i)                
            no_product.append(tmpProduct)
    
    df = df.drop(index=no_index)
    print(f'{len(no_index)} datos eliminados: No se encontraron los archivos de imagenes satelitales')
    return df , (no_index,no_product)

In [16]:
def obtenerDatos(filename):
    pdata = pd.read_csv(filename) 
    
    #Seleccionamos solo las columnas necesarias : precipitacion, Estacion (Longitud), Estacion (Latitud), Fecha (año-mes-dia-hora)
    pdataX = pdata.loc[:, ['dato','longitud', 'latitud', 'fecha']]

    #Quitamos los valores NA
    pdataX = pdataX[pdataX['dato'].notna()]

    #Definimos un solo tipo (str) pora asi poder convertirlo a tensor
    pdataX = pdataX.astype({"dato":str,"longitud":str, "latitud":  str, "fecha": str})
                
    #Barajeamos los datos
    pdataX = shuffle(pdataX)
    
    print(f'{len(pdataX)} datos leidos')
    return pdataX

In [49]:
def xyDataset(dataset,umbral, path_base,margen,products):
    x = []
    y = []
    i,j = 0.0 , []    
    for dato in dataset:       
        i,j =  leerImagenArea(dato,umbral, path_base,margen,products)
        x.append(i)
        y.append(j)
    x = np.asarray(x)
    y = np.asarray(y)
    return x,y
    

In [50]:
#@tf.function
def train_step(x,y,model,optimizer,loss_fn,train_acc_metric):    
    
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

In [51]:
#@tf.function
def test_step(x,y,model,val_acc_metric):      
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)

In [52]:
def entrenamiento(datasetList,umbral,model,path_base,margen,products, batch_size,train_size,epocas=2):  
    
    #Dividmos el dataset (Entrenamient - Validacion)
    train_dataset = tf.data.Dataset.from_tensor_slices(datasetList[:train_size])           
    val_dataset = tf.data.Dataset.from_tensor_slices(datasetList[train_size:])

    #Divimos en batchs los datasets
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
    val_dataset = val_dataset.batch(batch_size)
    

    #Definimos Variables del modelo (optmizador, funcion de loss, metricas, etc)
    optimizer = keras.optimizers.Adam(learning_rate=1e-3)        
    loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
    val_acc_metric = keras.metrics.SparseCategoricalAccuracy()
        
    #Entrenamos el modelo
    for epoch in range(epocas):    
        print("\nComienzo de la epoca %d" % (epoch,))
        start_time = time.time()
        
        for step, (datos) in enumerate(train_dataset):            
            #Obtenmos el verdadero dataset (valor, matriz) del batch X
            x_train_batch, y_train_batch =  xyDataset(datos,umbral, path_base,margen,products)  
            
            #Se obtiene el valor de perdida para el batch X
            loss_value = train_step(x_train_batch,y_train_batch,model,optimizer,loss_fn,train_acc_metric)
                        
            #Log every 200 batches.
            if step % 5 == 0:
                print(
                    "Training loss (for one batch) at step %d: %.4f"
                    % (step, float(loss_value))
                )
                print("Seen so far: %d samples" % ((step + 1) * batch_size))
        
        #Imprimimos y reiniciamos las metricas para una epoca
        train_acc = train_acc_metric.result()
        print("Training acc over epoch: %.4f" % (float(train_acc),))        
        train_acc_metric.reset_states()

        #Usamos el dataset de validacion para la validacion
        for (datos) in val_dataset:
            #Verdadero dataset
            x_val_batch, y_val_batch =  xyDataset(datos,umbral, path_base,margen,products)

            #Evaluamos
            test_step(x_val_batch, y_val_batch, model,val_acc_metric)

        #Imprimimos y reinciamos
        val_acc = val_acc_metric.result()
        print("Validation acc: %.4f" % (float(val_acc),))    
        val_acc_metric.reset_states()
        print("Time taken: %.2fs" % (time.time() - start_time))

In [22]:
#Path_base debe ser completo, se usará para comprobar si existen las imagenes satelitales descargadas
path_base = 'C:/Users/Shounen/Desktop/Ciclo XI/Tesis 2/GOES/'

#Productos de las imagenes satelitales (C08, C07 o C13, C02)
products = ['C08']

#El margen servira para recortar la imagen [alto, ancho] desde el punto de origen (estacion), esta en decimales
margen = [1,1]

#Batach -1 = entrenara con todo el dataset al mismo tiempo
batch = -1

In [24]:
#Leemos los datos del archivo
#Archivo de prueba contiene datos del 2019 del mes 01 y 02
dfOrignial = obtenerDatos('pruebasV2.csv')

336720 datos leidos


In [25]:
#Comprobamos si existen las imagenes/produtos por cada dato,
#caso contrario los borra de la lista
dfVerificado, (no_i,no_p) = comprobarFile(dfOrignial,products,path_base)

1444 datos eliminados: No se encontraron los archivos de imagenes satelitales


In [26]:
#Seleccionamos algunos para las pruebas
df = dfVerificado[0:1000]
datasetList = df.values.tolist()

#-Visualizacion
print(len(datasetList))
print(datasetList[0])

1000
['4.5', '-76.62455', '-11.13916', '2019-02-23-17']


In [45]:
#Creamos el modelo
dimOutput = 2
tempTensor =  tf.constant(datasetList[0])
imagenT, datoT = leerImagenArea(tempTensor, path_base,margen,products)
modelo = crearModelo(imagenT.shape[0],imagenT.shape[1],len(products),dimOutput)

Se creo un modelo con input ((110, 110, 1)) y output(2)
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 108, 108, 32)      320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 26, 26, 64)       0         
 2D)                                                             
                                                                 
 conv2d_11 (Conv2D)          (None, 24, 24, 64)        36928     
                                                                

In [55]:
#POR EL MOMENTO el batch_size debe poder dividirse entre la cantidad total del dataset (no residuo) 
batch_size = 100

#El Umbral esta en mm/h, igual que el dataset. Si supera este umbral se considera 1 (Extremo) sino 0 (no extremo)
umbral = 2.0

train_size = int(len(datasetList)*0.8)
epocas = 4

#Entrenamos con TODO el dataset              
entrenamiento(datasetList,umbral,modelo,path_base,margen,products, batch_size,train_size, epocas)


Comienzo de la epoca 0
Training loss (for one batch) at step 0: 1.7938
Seen so far: 100 samples
Training loss (for one batch) at step 5: 0.2696
Seen so far: 600 samples
Training acc over epoch: 0.8575
Validation acc: 0.9750
Time taken: 19.19s

Comienzo de la epoca 1
Training loss (for one batch) at step 0: 0.1679
Seen so far: 100 samples
Training loss (for one batch) at step 5: 0.2543
Seen so far: 600 samples
Training acc over epoch: 0.8525
Validation acc: 0.9750
Time taken: 19.59s

Comienzo de la epoca 2
Training loss (for one batch) at step 0: 0.2959
Seen so far: 100 samples
Training loss (for one batch) at step 5: 0.7532
Seen so far: 600 samples
Training acc over epoch: 0.9725
Validation acc: 0.0250
Time taken: 19.93s

Comienzo de la epoca 3
Training loss (for one batch) at step 0: 0.9669
Seen so far: 100 samples
Training loss (for one batch) at step 5: 0.6428
Seen so far: 600 samples
Training acc over epoch: 0.8550
Validation acc: 0.9750
Time taken: 23.08s


In [28]:
#Entrenamos con BATCH
#entrenamiento(modelo,datasetList,path_base,margen,products, 64 ,epocas=5)